# Task 2.4: Einsum Mastery

**Module:** 2 - Python for AI/ML  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand Einstein summation notation (einsum)
- [ ] Implement common operations using einsum
- [ ] Build attention score computation using einsum
- [ ] Compare einsum with explicit implementations

---

## 📚 Prerequisites

- Completed: Tasks 2.1-2.3
- Knowledge of: Matrix multiplication, broadcasting

---

## 🌍 Real-World Context

**Why learn einsum?**

Einsum is the Swiss Army knife of tensor operations:
- **Transformers** use einsum for attention computation
- **PyTorch and TensorFlow** both support einsum natively
- **Research papers** often use Einstein notation

Once you master einsum, you can:
- Read cutting-edge ML research more easily
- Write cleaner, more expressive code
- Implement complex operations in a single line

---

## 🧒 ELI5: What is Einsum?

> **Imagine you're giving directions for a treasure hunt...** 🗺️
>
> Instead of saying:
> "Take the first row of map A, multiply it by the first column of map B,
> add all the numbers, put it in position (1,1) of the result... repeat for every position"
>
> You can just say:
> "For each i,j in the result, sum over k: A[i,k] × B[k,j]"
>
> Or even shorter: `'ik,kj->ij'`
>
> **In AI terms:** Einsum lets you describe tensor operations using index notation.
> Indices that appear on both sides get matched, indices that disappear get summed over.

---

In [ ]:
import numpy as np
import time

print(f"NumPy version: {np.__version__}")
print(f"\n{'='*50}")
print("Welcome to Einsum Mastery! 🧮")
print(f"{'='*50}")

def time_function(func, *args, n_runs=5):
    """Time a function over multiple runs."""
    times = []
    for _ in range(n_runs):
        start = time.perf_counter()
        result = func(*args)
        times.append(time.perf_counter() - start)
    return np.mean(times), result

---

## Part 1: Einsum Basics

### The Einsum Notation

```
np.einsum('input_indices -> output_indices', arrays)
```

Rules:
1. Each input array gets labels for its dimensions
2. Indices that appear in inputs but not output are **summed over**
3. Indices in output define the result shape

In [ ]:
# Let's start simple!

# Example 1: Sum all elements
A = np.array([[1, 2], [3, 4]])

# 'ij->' means: take all i,j indices, no output indices = sum everything
result_einsum = np.einsum('ij->', A)
result_sum = np.sum(A)

print("Matrix A:")
print(A)
print(f"\nSum all elements:")
print(f"  np.einsum('ij->', A) = {result_einsum}")
print(f"  np.sum(A)            = {result_sum}")

In [ ]:
# Example 2: Sum along an axis

# Sum over j (columns) = row sums
row_sums_einsum = np.einsum('ij->i', A)
row_sums_numpy = np.sum(A, axis=1)

print(f"Sum over columns (row sums):")
print(f"  np.einsum('ij->i', A) = {row_sums_einsum}")
print(f"  np.sum(A, axis=1)     = {row_sums_numpy}")

# Sum over i (rows) = column sums
col_sums_einsum = np.einsum('ij->j', A)
col_sums_numpy = np.sum(A, axis=0)

print(f"\nSum over rows (column sums):")
print(f"  np.einsum('ij->j', A) = {col_sums_einsum}")
print(f"  np.sum(A, axis=0)     = {col_sums_numpy}")

In [ ]:
# Example 3: Transpose

# 'ij->ji' swaps the indices
transpose_einsum = np.einsum('ij->ji', A)
transpose_numpy = A.T

print("Original A:")
print(A)
print(f"\nTranspose with einsum ('ij->ji'):")
print(transpose_einsum)
print(f"\nMatches A.T? {np.allclose(transpose_einsum, transpose_numpy)}")

In [ ]:
# Example 4: Diagonal elements

A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# 'ii->i' extracts diagonal (where row index = column index)
diag_einsum = np.einsum('ii->i', A)
diag_numpy = np.diag(A)

print("Matrix A:")
print(A)
print(f"\nDiagonal with einsum ('ii->i'): {diag_einsum}")
print(f"Diagonal with np.diag:          {diag_numpy}")

In [ ]:
# Example 5: Trace (sum of diagonal)

# 'ii->' means: take diagonal elements (ii), sum them all (no output)
trace_einsum = np.einsum('ii->', A)
trace_numpy = np.trace(A)

print(f"Trace with einsum ('ii->'): {trace_einsum}")
print(f"Trace with np.trace:        {trace_numpy}")
print(f"\n💡 The trace is 1+5+9 = {1+5+9}")

### 🔍 Einsum Pattern Cheat Sheet (Part 1)

| Operation | Einsum | NumPy Equivalent |
|-----------|--------|------------------|
| Sum all | `'ij->'` | `np.sum(A)` |
| Row sums | `'ij->i'` | `np.sum(A, axis=1)` |
| Col sums | `'ij->j'` | `np.sum(A, axis=0)` |
| Transpose | `'ij->ji'` | `A.T` |
| Diagonal | `'ii->i'` | `np.diag(A)` |
| Trace | `'ii->'` | `np.trace(A)` |

---

## Part 2: Two-Array Operations

In [ ]:
# Example 6: Dot product (vectors)

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

# 'i,i->' means: multiply corresponding elements (same index i), sum all
dot_einsum = np.einsum('i,i->', a, b)
dot_numpy = np.dot(a, b)

print(f"a = {a}")
print(f"b = {b}")
print(f"\nDot product with einsum ('i,i->'): {dot_einsum}")
print(f"Dot product with np.dot:           {dot_numpy}")
print(f"\n💡 1×4 + 2×5 + 3×6 = {1*4 + 2*5 + 3*6}")

In [ ]:
# Example 7: Outer product

# 'i,j->ij' means: for each combination of i and j, multiply a[i] × b[j]
outer_einsum = np.einsum('i,j->ij', a, b)
outer_numpy = np.outer(a, b)

print(f"Outer product with einsum ('i,j->ij'):")
print(outer_einsum)
print(f"\nMatches np.outer? {np.allclose(outer_einsum, outer_numpy)}")

In [ ]:
# Example 8: Matrix multiplication!

A = np.random.randn(3, 4)
B = np.random.randn(4, 5)

# 'ik,kj->ij' means:
# - A has indices (i,k)
# - B has indices (k,j)  
# - k appears in both but not in output → sum over k
# - Result has indices (i,j)
matmul_einsum = np.einsum('ik,kj->ij', A, B)
matmul_numpy = A @ B

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"Result shape: {matmul_einsum.shape}")
print(f"\nMatches A @ B? {np.allclose(matmul_einsum, matmul_numpy)}")

In [ ]:
# Example 9: Element-wise multiplication

A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

# 'ij,ij->ij' means: same indices in, same indices out = element-wise
hadamard_einsum = np.einsum('ij,ij->ij', A, B)
hadamard_numpy = A * B

print("Element-wise multiplication:")
print(hadamard_einsum)
print(f"\nMatches A * B? {np.allclose(hadamard_einsum, hadamard_numpy)}")

### 🔍 Einsum Pattern Cheat Sheet (Part 2)

| Operation | Einsum | NumPy Equivalent |
|-----------|--------|------------------|
| Dot product | `'i,i->'` | `np.dot(a, b)` |
| Outer product | `'i,j->ij'` | `np.outer(a, b)` |
| Matrix multiply | `'ik,kj->ij'` | `A @ B` |
| Element-wise | `'ij,ij->ij'` | `A * B` |
| Matrix-vector | `'ij,j->i'` | `A @ v` |

---

## Part 3: Batch Operations (This is where it gets powerful!)

In [ ]:
# Example 10: Batch matrix multiplication

batch_size = 32
A = np.random.randn(batch_size, 64, 128)  # 32 matrices of 64×128
B = np.random.randn(batch_size, 128, 64)  # 32 matrices of 128×64

# 'bik,bkj->bij' means:
# - b is batch dimension (preserved)
# - ik,kj->ij is matrix multiply (k summed over)
batch_matmul_einsum = np.einsum('bik,bkj->bij', A, B)
batch_matmul_numpy = A @ B

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"Result shape: {batch_matmul_einsum.shape}")
print(f"\nMatches A @ B? {np.allclose(batch_matmul_einsum, batch_matmul_numpy)}")

In [ ]:
# Example 11: Batch dot product (useful for similarity scores)

# Two sets of vectors: compute dot product for corresponding pairs
queries = np.random.randn(32, 128)  # 32 query vectors
keys = np.random.randn(32, 128)     # 32 key vectors

# 'bi,bi->b' means: for each b, compute dot product of vectors
batch_dot_einsum = np.einsum('bi,bi->b', queries, keys)
batch_dot_loop = np.array([np.dot(q, k) for q, k in zip(queries, keys)])

print(f"Queries shape: {queries.shape}")
print(f"Keys shape: {keys.shape}")
print(f"Result shape: {batch_dot_einsum.shape}")
print(f"\nFirst 5 dot products: {batch_dot_einsum[:5].round(2)}")
print(f"Matches loop? {np.allclose(batch_dot_einsum, batch_dot_loop)}")

In [ ]:
# Example 12: Broadcasting in einsum

# Single matrix applied to batch of vectors
W = np.random.randn(256, 128)       # Weight matrix
X = np.random.randn(32, 128)        # Batch of 32 input vectors

# 'oi,bi->bo' means:
# - W has indices (o,i)
# - X has indices (b,i)
# - i summed over, result is (b,o)
# This is X @ W.T
result_einsum = np.einsum('oi,bi->bo', W, X)
result_numpy = X @ W.T

print(f"W shape: {W.shape}")
print(f"X shape: {X.shape}")
print(f"Result shape: {result_einsum.shape}")
print(f"\nMatches X @ W.T? {np.allclose(result_einsum, result_numpy)}")

---

## Part 4: Attention Score Computation 🔥

### 🧒 ELI5: What is Attention?

> **Imagine reading a story...** 📖
>
> When you read "The cat sat on the mat because **it** was tired", 
> your brain automatically knows "it" refers to "cat", not "mat".
>
> **Attention** gives AI the same ability! For each word, it computes
> "how much should I pay attention to every other word?"
>
> The formula: $\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$

In [ ]:
# Attention dimensions (like in a Transformer)
batch_size = 8
num_heads = 12
seq_length = 64
head_dim = 64

# Q, K, V tensors: (batch, heads, seq_len, head_dim)
Q = np.random.randn(batch_size, num_heads, seq_length, head_dim).astype(np.float32)
K = np.random.randn(batch_size, num_heads, seq_length, head_dim).astype(np.float32)
V = np.random.randn(batch_size, num_heads, seq_length, head_dim).astype(np.float32)

print(f"Q shape: {Q.shape}")
print(f"K shape: {K.shape}")
print(f"V shape: {V.shape}")
print(f"\n💡 This represents {batch_size} sequences, {num_heads} attention heads,")
print(f"   each sequence has {seq_length} tokens, each head dimension is {head_dim}")

In [ ]:
# Step 1: Compute attention scores = Q @ K.T
# Shape: (batch, heads, seq, seq)

# Using einsum: 'bhsd,bhtd->bhst'
# - Q: (b)atch, (h)eads, (s)eq, (d)im
# - K: (b)atch, (h)eads, (t)arget_seq, (d)im
# - d summed over → result is (b, h, s, t)
scores_einsum = np.einsum('bhsd,bhtd->bhst', Q, K)

# Using matmul (need to transpose K's last two dims)
scores_matmul = Q @ K.transpose(0, 1, 3, 2)  # Transpose last two dims

print(f"Attention scores shape: {scores_einsum.shape}")
print(f"Matches matmul version? {np.allclose(scores_einsum, scores_matmul)}")

In [ ]:
# Step 2: Scale by sqrt(d_k)
scale = np.sqrt(head_dim)
scaled_scores = scores_einsum / scale

print(f"Scale factor: {scale}")
print(f"Scaled scores shape: {scaled_scores.shape}")

In [ ]:
# Step 3: Apply softmax
def softmax(x, axis=-1):
    """Numerically stable softmax."""
    x_max = x.max(axis=axis, keepdims=True)
    exp_x = np.exp(x - x_max)
    return exp_x / exp_x.sum(axis=axis, keepdims=True)

attention_weights = softmax(scaled_scores, axis=-1)

print(f"Attention weights shape: {attention_weights.shape}")
print(f"Weights sum to 1? {np.allclose(attention_weights.sum(axis=-1), 1.0)}")

In [ ]:
# Step 4: Apply attention to values = Attn @ V
# Shape: (batch, heads, seq, head_dim)

# Using einsum: 'bhst,bhtd->bhsd'
# - Attention: (b, h, s, t)
# - V: (b, h, t, d)
# - t summed over → result is (b, h, s, d)
output_einsum = np.einsum('bhst,bhtd->bhsd', attention_weights, V)

# Using matmul
output_matmul = attention_weights @ V

print(f"Output shape: {output_einsum.shape}")
print(f"Matches matmul version? {np.allclose(output_einsum, output_matmul)}")

In [ ]:
# Complete attention function!

def attention_einsum(Q, K, V):
    """
    Compute scaled dot-product attention using einsum.
    
    Args:
        Q: Query tensor (batch, heads, seq, dim)
        K: Key tensor (batch, heads, seq, dim)
        V: Value tensor (batch, heads, seq, dim)
    
    Returns:
        Output tensor (batch, heads, seq, dim)
    """
    d_k = Q.shape[-1]
    
    # Compute attention scores: Q @ K.T / sqrt(d_k)
    scores = np.einsum('bhsd,bhtd->bhst', Q, K) / np.sqrt(d_k)
    
    # Apply softmax
    weights = softmax(scores, axis=-1)
    
    # Apply to values
    output = np.einsum('bhst,bhtd->bhsd', weights, V)
    
    return output, weights

# Test it!
output, weights = attention_einsum(Q, K, V)
print(f"✅ Attention output shape: {output.shape}")
print(f"   Attention weights shape: {weights.shape}")

### Timing Comparison

In [ ]:
def attention_matmul(Q, K, V):
    """Attention using standard matmul."""
    d_k = Q.shape[-1]
    scores = (Q @ K.transpose(0, 1, 3, 2)) / np.sqrt(d_k)
    weights = softmax(scores, axis=-1)
    output = weights @ V
    return output, weights

# Time both versions
einsum_time, _ = time_function(attention_einsum, Q, K, V, n_runs=10)
matmul_time, _ = time_function(attention_matmul, Q, K, V, n_runs=10)

print(f"Einsum version: {einsum_time*1000:.2f} ms")
print(f"Matmul version: {matmul_time*1000:.2f} ms")
print(f"\n💡 Both are similar because NumPy optimizes both patterns!")

---

## ✋ Try It Yourself: Exercises

### Exercise 1: Batch Outer Product

Compute the outer product for each pair of vectors in a batch.

- Input: Two tensors of shape `(batch, dim)`
- Output: Shape `(batch, dim, dim)`

In [ ]:
# Exercise 1
np.random.seed(42)
a = np.random.randn(16, 64)  # 16 vectors of dim 64
b = np.random.randn(16, 64)  # 16 vectors of dim 64

# TODO: Use einsum to compute batch outer product
# batch_outer = np.einsum('??,??->???', a, b)

# Uncomment to verify:
# print(f"Result shape: {batch_outer.shape}")  # Should be (16, 64, 64)
# expected = np.array([np.outer(a[i], b[i]) for i in range(16)])
# print(f"Correct? {np.allclose(batch_outer, expected)}")

<details>
<summary>💡 Hint for Exercise 1</summary>

For outer product of vectors: `'i,j->ij'`

Add batch dimension b: `'bi,bj->bij'`

</details>

### Exercise 2: Bilinear Form

Compute $x^T A y$ for batches of vectors and a shared matrix.

- x: shape `(batch, m)`
- A: shape `(m, n)` (shared across batch)
- y: shape `(batch, n)`
- Result: shape `(batch,)`

In [ ]:
# Exercise 2
np.random.seed(42)
x = np.random.randn(32, 64)
A = np.random.randn(64, 128)
y = np.random.randn(32, 128)

# TODO: Compute x^T A y using einsum
# bilinear = np.einsum('...', x, A, y)

# Uncomment to verify:
# print(f"Result shape: {bilinear.shape}")  # Should be (32,)
# expected = np.array([x[i] @ A @ y[i] for i in range(32)])
# print(f"Correct? {np.allclose(bilinear, expected)}")

<details>
<summary>💡 Hint for Exercise 2</summary>

Break it down:
1. x has indices (b, m)
2. A has indices (m, n)
3. y has indices (b, n)
4. m and n should be summed over

Answer: `'bm,mn,bn->b'`

</details>

---

## ⚠️ Common Mistakes

### Mistake 1: Wrong index order

In [ ]:
A = np.random.randn(3, 4)
B = np.random.randn(4, 5)

# ❌ Wrong: indices don't match for matmul
# result = np.einsum('ij,jk->ik', A, B)  # This would work
# result = np.einsum('ij,kj->ik', A, B)  # This is different! (A @ B.T)

# ✅ Right: match the shared dimension
result = np.einsum('ij,jk->ik', A, B)  # Standard matmul
print(f"Result shape: {result.shape}")
print(f"\n💡 Always check which indices are being summed over!")

### Mistake 2: Forgetting optimize=True for large operations

In [ ]:
# For complex einsum with many tensors, use optimize=True
# This lets NumPy find the optimal contraction order

A = np.random.randn(10, 20)
B = np.random.randn(20, 30)
C = np.random.randn(30, 40)

# ✅ Good: let NumPy optimize
result = np.einsum('ij,jk,kl->il', A, B, C, optimize=True)
print(f"Result shape: {result.shape}")
print(f"\n💡 optimize=True is important for 3+ tensor contractions!")

---

## 🎉 Checkpoint

You've learned:
- ✅ Basic einsum operations (sum, transpose, diagonal, trace)
- ✅ Two-tensor operations (dot, outer, matmul)
- ✅ Batch operations for ML workloads
- ✅ Implementing attention with einsum

## 📋 Complete Einsum Reference

| Operation | Einsum | Explicit |
|-----------|--------|----------|
| Sum | `'ij->'` | `np.sum(A)` |
| Transpose | `'ij->ji'` | `A.T` |
| Diagonal | `'ii->i'` | `np.diag(A)` |
| Trace | `'ii->'` | `np.trace(A)` |
| Dot product | `'i,i->'` | `np.dot(a,b)` |
| Outer product | `'i,j->ij'` | `np.outer(a,b)` |
| Matrix multiply | `'ik,kj->ij'` | `A @ B` |
| Batch matmul | `'bik,bkj->bij'` | `A @ B` |
| Attention scores | `'bhsd,bhtd->bhst'` | `Q @ K.T` |

---

## 📖 Further Reading

- [Einstein Summation in NumPy](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html)
- [Einsum is All You Need (Blog)](https://rockt.github.io/2018/04/30/einsum)
- [Attention Is All You Need (Paper)](https://arxiv.org/abs/1706.03762)

---

## 🧹 Cleanup

In [ ]:
import gc

del Q, K, V, output, weights
gc.collect()

print("✅ Memory cleaned up!")
print("\n🎉 Congratulations! You've mastered einsum!")
print("   Next up: Task 2.5 - Profiling Exercise")